<img src="https://toppng.com/uploads/preview/linkedin-logo-png-photo-116602552293wtc4qogql.png" width="20" height="20" /> [Bharath Hemachandran](https://www.linkedin.com/in/bharath-hemachandran/)

# 🔌 Phase 2: Groq + One MCP (Hugging Face)

Add **one MCP server** so the model can use **tools** (e.g. search Hugging Face). You'll learn what MCP is, why it's needed, how to choose and find MCP servers, and end with a **real-world application**.

<div style="background: #fce4ec; padding: 14px; border-radius: 8px; border-left: 4px solid #c2185b;">
<strong>🎯 What you'll do:</strong> Understand MCP servers (need, evaluation, examples, where to find them), call Groq with one MCP tool, and build a practical MCP-based workflow.
</div>

### 📋 Notebook objective (table of contents)

This notebook covers:
- **Setup** — OpenAI client, API keys (GROQ, optional HF)
- **What is MCP?** — Model Context Protocol; servers, tools, resources
- **Why do we need MCP?** — LLMs can't access tools/data natively; MCP standardizes it
- **What can you do with an MCP server?** — Tools, resources, prompts
- **How to evaluate what MCP servers you need** — Task → capabilities → pick server
- **Examples with MCP** — Search, read, call APIs; Hugging Face example
- **How to find MCP servers** — Registry, GitHub, docs, Groq/Cursor
- **Define the MCP tool** — type, server URL, require_approval: "never"
- **Call Groq with the MCP tool** — One request; model uses tool and returns text
- **Output items** — Types in <code>response.output</code> (e.g. mcp_call)
- **Real-world application** — Practical MCP workflow (e.g. model discovery + summary)
- **Exercises** — Why MCP, matching task to server, changing the real-world prompt
- **Additional reading** — MCP spec and Groq docs


## 🔧 Setup (run once)

Install **openai**. On Colab, run this cell first.

In [ ]:
!pip install -q openai

### 🔑 API keys

- **GROQ_API_KEY** — required. Get it at [console.groq.com](https://console.groq.com/keys).<br>
- **HF_TOKEN** (optional) — for Hugging Face MCP; improves rate limits. Get it at [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens).

In [ ]:
import os
from getpass import getpass

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass("Paste your GROQ_API_KEY: ")
if not os.environ.get("HF_TOKEN"):
    tok = getpass("Paste your HF_TOKEN (or press Enter to skip): ")
    if tok:
        os.environ["HF_TOKEN"] = tok

from openai import OpenAI

def get_groq_client():
    return OpenAI(
        api_key=os.environ["GROQ_API_KEY"],
        base_url="https://api.groq.com/openai/v1",
    )

HF_TOKEN = os.environ.get("HF_TOKEN")
print("✅ Groq client ready." if os.environ.get("GROQ_API_KEY") else "Need GROQ_API_KEY.")

## What is the Model Context Protocol (MCP)?

**MCP** is an **open protocol** that lets LLM applications (like Groq, Cursor, or Claude) talk to **external tools and data** in a standard way.

- **MCP server** — A service that exposes **tools** (actions the model can call, e.g. “search Hugging Face”), **resources** (read-only data, e.g. files or APIs), and optionally **prompts** (predefined prompt templates). The server runs somewhere (your machine, a cloud URL) and speaks the MCP protocol (JSON-RPC over stdio or HTTP/SSE).
- **MCP client** — The application that has the model (e.g. Groq API, Cursor). It connects to one or more MCP servers, sends the user’s prompt, and when the model decides to use a tool, the client calls that tool on the server and returns the result to the model.
- **Tool** — One capability the server provides (e.g. “search_models”, “read_file”). The model sees tool names and descriptions, chooses when to call them, and receives structured results.

So: **you add an MCP server** (by URL or config) to your client; the **model then can use that server’s tools** during a conversation, without you hand-writing glue code for each integration.

## Why do we need MCP?

LLMs only see **text**. They can’t open files, call APIs, or search the web by themselves. To do real tasks (search models, read docs, run code), something has to:

1. **Expose** those actions as callable “tools” with clear names and parameters.
2. **Run** the tool when the model asks and **return** the result as text (or structured data) back to the model.
3. Do this in a **consistent way** across many apps and providers.

**Without a standard:** Every app (Cursor, Groq, Claude, etc.) would invent its own way to plug in tools → more work for developers and fewer reusable integrations.

**With MCP:** One protocol for “what tools exist” and “call this tool with these arguments.” Servers (Hugging Face, filesystem, Slack, etc.) implement MCP once; any MCP-capable client can use them. So you get **one way to add capabilities** (search, read, execute) to any model that supports MCP.

## What can you do with an MCP server?

MCP servers expose three kinds of things (depending on the server):

| Capability | What it is | Example |
|------------|------------|---------|
| **Tools** | Actions the model can invoke (with arguments). The server runs the action and returns a result. | Search Hugging Face, run a script, create a calendar event, query a database. |
| **Resources** | Read-only data the model can read (by URI). | Read a file, fetch a URL, list directory contents. |
| **Prompts** | Predefined prompt templates the client can offer (e.g. “Summarize this doc”). | “Code review”, “Explain this error”, “Turn notes into a ticket”. |

In this notebook we use **tools** only (e.g. Hugging Face search). The model sees the tool list, chooses when to call one, and the Groq API sends that call to the MCP server and feeds the result back to the model so it can continue the reply.

## How to evaluate what MCP servers you need

1. **Define the task** — What should the model do? (e.g. “find a model on Hugging Face”, “read my README”, “search the web”, “run a linter”.)
2. **Map to capabilities** — Does the task need **search**, **read a file**, **call an API**, **run code**, **access a database**, etc.?
3. **Match to servers** — Look for MCP servers that expose those capabilities (e.g. Hugging Face MCP for model search, filesystem MCP for local files, custom server for your API).
4. **Check auth and environment** — Does the server need an API key (e.g. HF_TOKEN), or run only locally? Groq can call servers that are reachable by URL (e.g. https://huggingface.co/mcp).
5. **Start minimal** — Add one server, test with a few prompts, then add more servers if you need more capabilities.

**Example:** “I want the model to recommend a Hugging Face model for sentiment analysis” → need **search/list models** → use **Hugging Face MCP** (exposes model search). No need for filesystem or database MCP unless the task also involves local files or DB.

## Examples of things to do with an MCP server

| Goal | MCP server (example) | What the model can do |
|------|----------------------|------------------------|
| **Find / compare models** | Hugging Face MCP | Search models, read model cards, list trending or task-specific models. |
| **Read local files** | Filesystem MCP | Read files, list dirs; e.g. “summarize this README”, “what’s in this folder?” |
| **Search the web** | Web search MCP (if available) | Search and get snippets; e.g. “latest docs for library X”. |
| **Use your API** | Custom MCP server | Expose your API as tools; model calls endpoints and uses responses in the reply. |
| **Databases** | DB MCP (e.g. Postgres) | Query (read-only or controlled writes); e.g. “how many users signed up this week?” |
| **Slack / Notion / Google** | Official or community MCPs | Post messages, read/write docs, create calendar events from natural language. |

In this notebook we use **Hugging Face MCP**: the model can search models, read cards, and answer questions like “What models are trending?” or “Suggest a model for translation.”

## How to find MCP servers

- **Official / vendor registries** — [modelcontextprotocol.io](https://modelcontextprotocol.io) and Anthropic’s MCP docs list servers (e.g. Google Drive, Slack, GitHub). Groq and Cursor docs also mention supported or recommended MCPs.
- **GitHub** — Search for “MCP server” or “model context protocol”; many repos implement servers for Hugging Face, filesystem, Postgres, custom APIs, etc. Check the README for the server URL or how to run it.
- **Hugging Face** — The Hugging Face MCP we use here is documented at [huggingface.co/mcp](https://huggingface.co/mcp); you use the URL `https://huggingface.co/mcp` and optionally pass `HF_TOKEN` for higher rate limits.
- **Build your own** — Implement the MCP spec (tools/resources/prompts) in any language; expose it over HTTP/SSE or stdio so the client can connect. Useful for internal APIs or custom data sources.

**Tip:** Prefer servers that document their **tools** (names, arguments, return shape) and **auth** (API key, OAuth, or none) so you know what the model can do and how to configure it.

## 🔧 Define the MCP tool

One tool entry: <code>type: "mcp"</code>, server URL, and <code>require_approval: "never"</code> so the run doesn't wait for approval.

In [ ]:
mcp_tool = {
    "type": "mcp",
    "server_label": "Huggingface",
    "server_url": "https://huggingface.co/mcp",
    "server_description": "Search and access AI models from Hugging Face",
    "require_approval": "never",
}
if HF_TOKEN:
    mcp_tool["headers"] = {"Authorization": f"Bearer {HF_TOKEN}"}

print("MCP tool: Hugging Face (require_approval: never)")

## 🚀 Call Groq with the MCP tool

We use a model that supports tools (e.g. <code>openai/gpt-oss-120b</code>). The model may issue **tool calls**; with <code>require_approval: "never"</code> they run automatically.

In [ ]:
client = get_groq_client()

response = client.responses.create(
    model="openai/gpt-oss-120b",
    input="What models are trending on Hugging Face? List up to 3.",
    tools=[mcp_tool],
)

print("--- output_text ---")
print(response.output_text)

## 📦 Output items (types)

The <code>response.output</code> list can contain text and tool-related items (e.g. <code>mcp_call</code>).

In [ ]:
print("--- output item types ---")
for item in response.output:
    t = getattr(item, "type", item)
    print(t)

## Real-world application: model discovery and recommendation

**Use case:** A developer wants to pick a **Hugging Face model for a task** (e.g. sentiment analysis or translation) without browsing the Hub manually. The model uses the **Hugging Face MCP** to search and read model info, then returns a short recommendation.

**Flow:** You ask in natural language (e.g. “Find a good model for sentiment analysis in English and summarize why it’s a good fit”). The LLM:

1. Calls the Hugging Face MCP **tool(s)** to search or list models (e.g. by task or name).
2. Receives tool results (model names, cards, metrics).
3. Synthesizes a short recommendation and explanation in plain language.

Below we run one such request: the model uses the MCP server to answer a practical “find and recommend” question.

In [ ]:
# Real-world application: use MCP to find and recommend a model for a task
client = get_groq_client()

# Same MCP tool as before (Hugging Face)
mcp_tool = {
    "type": "mcp",
    "server_label": "Huggingface",
    "server_url": "https://huggingface.co/mcp",
    "server_description": "Search and access AI models from Hugging Face",
    "require_approval": "never",
}
if os.environ.get("HF_TOKEN"):
    mcp_tool["headers"] = {"Authorization": f"Bearer {os.environ['HF_TOKEN']}"}

# Practical prompt: find a model for a task and summarize why it fits
prompt = (
    "I need a model for sentiment analysis on English product reviews. "
    "Use Hugging Face to find a suitable model, then in one short paragraph "
    "recommend one and explain why it's a good fit (e.g. task, license, popularity)."
)

resp = client.responses.create(
    model="openai/gpt-oss-120b",
    input=prompt,
    tools=[mcp_tool],
)

print("--- Real-world application: model recommendation ---")
print(resp.output_text)
print("\n--- Tool calls used (mcp_call items) ---")
for item in resp.output:
    if getattr(item, "type", None) == "mcp_call":
        print(f"  Tool: {getattr(item, 'name', '?')}")

In [ ]:
print("✅ Phase 2 complete. Next: Phase 3 (multi-step MCP agent).")

## ✏️ Exercises

*Use only what you learned in this phase (MCP, tools, server choice, Groq + MCP call, mcp_call).*

1. **Why MCP?**  
   In one sentence, why can't an LLM "search Hugging Face" by itself—i.e. without an MCP server or similar—and what does the MCP server provide?

2. **Matching task to server**  
   Your task is: *"Read a local README file and summarize it in three bullet points."* What **capability** do you need (e.g. read file, search web, call API)? What **kind** of MCP server would you look for, and what would it expose (tools, resources, or both)?

3. **Change the real-world prompt**  
   In the "Real-world application" section, change the prompt so the user asks for a Hugging Face model for **image classification** (instead of sentiment analysis). Run the cell and compare: does the model use the MCP tool? What models does it recommend? Write one sentence on what you observed.

## 📚 Additional reading

**YouTube (verified)**  
- [The Model Context Protocol (MCP)](https://www.youtube.com/watch?v=CQywdSdi5iA) — Anthropic: MCP intro and why it matters.  
- [Model Context Protocol, clearly explained](https://www.youtube.com/watch?v=7j_NE6Pjv-E) — Why MCP matters for connecting AI to tools.

**Blogs (popular)**  
- [Model Context Protocol – Specification](https://modelcontextprotocol.io/specification/latest) — Official MCP spec: tools, resources, lifecycle.  
- [Groq API Reference](https://console.groq.com/docs) — Groq docs: Responses API, tools, MCP connectors.